In [1]:
import pandas as pd
import numpy as np
import requests

In [5]:
# !pip install bs4
from bs4 import BeautifulSoup

# Get Movies Address According To Alphabets

In [231]:
def get_movies_link():
    url = "https://en.wikipedia.org/wiki/Lists_of_films#By_country"
    res =  requests.get(url)
    soup = BeautifulSoup(str(res.content).replace('<br>', '\n'), 'html.parser')
    movies_by_alpha = []
    for row in soup.find(class_ = 'wikitable').find('tbody').find_all('tr')[1:]:
        for src in row.find_all('td'):
            movies_by_alpha.append(src.a.get('href'))
    return sorted(list(set(['https://en.wikipedia.org'+link for link in movies_by_alpha])))

In [232]:
get_movies_link()

['https://en.wikipedia.org/wiki/List_of_films:_A',
 'https://en.wikipedia.org/wiki/List_of_films:_B',
 'https://en.wikipedia.org/wiki/List_of_films:_C',
 'https://en.wikipedia.org/wiki/List_of_films:_D',
 'https://en.wikipedia.org/wiki/List_of_films:_E',
 'https://en.wikipedia.org/wiki/List_of_films:_F',
 'https://en.wikipedia.org/wiki/List_of_films:_G',
 'https://en.wikipedia.org/wiki/List_of_films:_H',
 'https://en.wikipedia.org/wiki/List_of_films:_I',
 'https://en.wikipedia.org/wiki/List_of_films:_J%E2%80%93K#J',
 'https://en.wikipedia.org/wiki/List_of_films:_J%E2%80%93K#K',
 'https://en.wikipedia.org/wiki/List_of_films:_L',
 'https://en.wikipedia.org/wiki/List_of_films:_M',
 'https://en.wikipedia.org/wiki/List_of_films:_N%E2%80%93O#N',
 'https://en.wikipedia.org/wiki/List_of_films:_P',
 'https://en.wikipedia.org/wiki/List_of_films:_Q%E2%80%93R#Q',
 'https://en.wikipedia.org/wiki/List_of_films:_Q%E2%80%93R#R',
 'https://en.wikipedia.org/wiki/List_of_films:_S',
 'https://en.wikipedia

# Get Movie Title And URL

In [212]:
def get_moves_list(url):
    base_url = "https://en.wikipedia.org"
    response = requests.get(url)
    soup = BeautifulSoup(str(response.content).replace('<br>', '\n'), 'html.parser')
    movies_list = []
    for section in soup.find_all(class_='div-col'):
        for item in section.ul.find_all('li'):
            try:
                if item.i.a.has_attr('href'):
                    url = item.i.a.get('href')
                    name_year = item.text
                    movies_list.append([base_url+url,name_year])
            except:
                for film in item.find_all('a'):
                    if film.has_attr('href'):
                        url = film.get('href')
                        name_year = film.get('title')
                        movies_list.append([base_url+url,name_year])
    return movies_list

In [233]:
get_moves_list("https://en.wikipedia.org/wiki/List_of_films:_A")

[['https://en.wikipedia.org/wiki/A_(Kannada_film)', 'A (1998)'],
 ['https://en.wikipedia.org/wiki/A_Aa_E_Ee_(2009_Tamil_film)',
  'A Aa E Ee (2009, Tamil)'],
 ['https://en.wikipedia.org/wiki/A_Aa_E_Ee_(2009_Telugu_film)',
  'A Aa E Ee (2009, Telugu)'],
 ['https://en.wikipedia.org/wiki/A-ge-man', 'A-ge-man (1990)'],
 ['https://en.wikipedia.org/wiki/A-Haunting_We_Will_Go_(1942_film)',
  'A-Haunting We Will Go (1942)'],
 ['https://en.wikipedia.org/wiki/A_Rodeo_Film', 'A Rodeo Film (2021)'],
 ['https://en.wikipedia.org/wiki/The_A-Team_(film)', 'The A-Team (2010)'],
 ['https://en.wikipedia.org/wiki/A.K.G._(film)', 'A.K.G. (2007)'],
 ['https://en.wikipedia.org/wiki/Aa_Ab_Laut_Chalen',
  'Aa Ab Laut Chalen (1999)'],
 ['https://en.wikipedia.org/wiki/Aa_Chithrashalabham_Parannotte',
  'Aa Chithrashalabham Parannotte (1970)'],
 ['https://en.wikipedia.org/wiki/Aa_Dekhen_Zara', 'Aa Dekhen Zara (2009)'],
 ['https://en.wikipedia.org/wiki/Aa_Dinagalu', 'Aa Dinagalu (2007)'],
 ['https://en.wikipedia.o

# Get Movie Details With URL

In [280]:
def get_movie_details(url,title,alpha_url)->dict:
    response = requests.get(url)
    soup = BeautifulSoup(str(response.content).replace('<br>', '\n'), 'html.parser')
    
    movie_info = {
        'AlphaURL':alpha_url,
        'Title':title,
        'MovieURL':url
    }
    print(title)
    if soup.find(class_ = "infobox vevent"):
        for row in soup.find(class_ = "infobox vevent").find_all('tr'):
            if row.td is not None:
                if row.find(class_='image'):
                    src = f"https:{row.find(class_='image').img.get('src')}"
                    movie_info.update({'Image':src})
                elif row.find(class_ = "thumbborder"):
                    src = f"https:{row.find(class_='thumbborder').get('poster')}"
                    movie_info.update({'Image':src})
                else:
                    if not row.th:
                        continue
                    header = row.th.text
                    if row.td.find('ul'):
                        data = []
                        for li in row.td.find('ul').find_all('li'):
                            data.append(li.text)
                    else:
                        data = row.td.text
                    movie_info.update({header:data})
    return movie_info

# How to create the DataFrame

In [262]:
movie_detail = get_movie_details('https://en.wikipedia.org//wiki/A_(Kannada_film)','A','https://en.wikipedia.org/wiki/List_of_films:_A')

In [268]:
# movie_detail

In [281]:
movie_data = []
movies = get_moves_list("https://en.wikipedia.org/wiki/List_of_films:_A")
for movie in movies:
    movie_detail = get_movie_details(movie[0], movie[1],'https://en.wikipedia.org/wiki/List_of_films:_A')
    movie_data.append(movie_detail)

A (1998)
A Aa E Ee (2009, Tamil)
A Aa E Ee (2009, Telugu)
A-ge-man (1990)
A-Haunting We Will Go (1942)
A Rodeo Film (2021)
The A-Team (2010)
A.K.G. (2007)
Aa Ab Laut Chalen (1999)
Aa Chithrashalabham Parannotte (1970)
Aa Dekhen Zara (2009)
Aa Dinagalu (2007)
Aa Divasam (1982)
Aa Gale Lag Jaa (1973 film)
Aa Gale Lag Jaa (1994 film)
Aa Jaa Sanam (1975)
Aa Naluguru (2004)
Aa Neram Alppa Dooram (1985)
Aa Nimisham (1977)
Aa Okkadu (2009)
Aa Okkati Adakku (1992)
Aa Raathri (1983)
Aaaaaaaah! (2015)
Aaaah (2014)
Aaag Hi Aag (1999)
Aaah (2001)
Aaahh Belinda (1986)
Aabhaasam (2018)
Aabhijathyam (1971)
Aabra Ka Daabra (2004)
Aabroo (1943)
Aabroo (1968)
Aabshar (1953)
Aachariyangal (2012)
Aacharya (2006)
Aacharyan (1993)
Aachi & Ssipak (2006)
Aada Paduchu (1967)
Aadaalla Majaka (1995)
Aadab Arz (1943)
Aadama Jaichomada (2014)
Aadamkhor (1986)
Aadanthe Ado Type (2013)
Aadarsa Kutumbam (1969)
Aadarsham (1993)
Aadat Se Majboor (1982)
Aadavaallu Meeku Joharlu (1981)
Aadavari Matalaku Arthale Verule (2

Acquitted for Having Committed the Deed (1992)
Across 110th Street (1972)
Across the Bridge (1957)
Across the Pacific (1942)
Across the River and Into the Trees (2022)
Across the Tracks (1991)
Across the Universe (2007)
Across the Wide Missouri (1951)
The Act of Killing (2012)
The Act in Question (1994)
Act of Valor (2012)
Act of Vengeance (1986)
Act of Violence
Act of Violence (1956 film)
Act of Violence (1959 film)
Action Jackson (1988 film)
Action Jackson (2014 film)
Action in the North Atlantic (1943)
Action Replayy (2010)
Action: The October Crisis of 1970 (1973)
Actor (1993 film)
Actor (2016 film)
Acts of Vengeance (2017)
Ad Astra (2019)
Ad Fundum (1993)
Ad-lib Night (2006)
Ada (1961 film)
Ada (2019 film)
Ada... A Way of Life (2010)
Ada Apa dengan Cinta?
Ada Apa dengan Cinta? (short film)
Ada Apa Dengan Cinta? 2 (2016)
Ada Apa Dengan Rina (2013)
Adada (1987)
Adada Enna Azhagu (2009)
Adagio (2000)
Adalat (1958 film)
Adalat (1976 film)
Adalat o Ekti Meye (1981)
Adalu Badalu (1979)


Advise & Consent (1962)
Adwa (1999)
Adyaksha in America (2019)
Ae Dil Hai Mushkil (2016)
Ae Fond Kiss... (2004)
Ae Jugara Krushna Sudama (2003)
Ae Kaash Ke Hum (2020)
Aedan (2018)
Aeg elada, aeg armastada (1976)
Aegri Somnia (2008)
Aelita (1924)
Aelita, Do Not Pester Men! (1988)
Aera! Aera! Aera! (1972)
Aerial Anarchists (1911)
An Aerial Joy Ride (1917)
An Aerial Joyride (1916)
Aerials (2016)
Aerograd (1935)
The Aeronauts (2019)
Aesop\'s Film Fables (1921)
Aetbaar (2004)
Aethiree (2004)
\xc3\x86on Flux (2005)
An Affair (1998)
The Affair of the Necklace (2001)
An Affair to Remember (1957)
Affair with a Stranger (1953)
The Affair of Susan (1935)
Affair in Trinidad (1952)
The Affairs of Anatol (1921)
Affairs of the Art (2021)
Affairs of Cappy Ricks (1937)
The Affairs of Cellini (1934)
The Affairs of Susan (1945)
Afflicted (2013)
The Afflicted (2011)
Affliction (1996 film)
Affliction (1997 film)
Afghan Breakdown (1990)
Aflatoon (1997)
Africa Screams (1949)
Africa: The Serengeti (1994)
Afri

Alice in Wonderland (1915 film)
Alice in Wonderland (1931 film)
Alice in Wonderland (1933 film)
Alice in Wonderland (1949 film)
Alice in Wonderland (1951 film)
Alice in Wonderland (1966 film)
Alice in Wonderland (1976 film)
Alice in Wonderland (1985 film)
Alice in Wonderland (1988 film)
Alice in Wonderland (1999 film)
Alice in Wonderland (2005 film)
Alice in Wonderland (2010 film)
Alice of Wonderland in Paris (1966)
Alice's Adventures in Wonderland (1910 film)
Alice's Adventures in Wonderland (1972 film)
Alice\'s Restaurant (1969)
Alice, Sweet Alice (1976)
Alicia (1974)
Alien series:\nAlien (1979)\nAliens (1986)\nAlien 3 (1992)\nAlien Resurrection (1997)\nPrometheus (2012)\nAlien: Covenant (2017)
Alien (1979)
Aliens (1986)
Alien 3 (1992)
Alien Resurrection (1997)
Prometheus (2012)
Alien: Covenant (2017)
Alien 2: On Earth (1980)
Alien Abduction (2005 film)
Alien Abduction (2014 film)
Alien Adventure (1999)
Alien Apocalypse (2005 TV)
Alien Autopsy (2006)
Alien Contamination (1980)
Alien 

The American (1927 film)
The American (2010 film)
American Animals (2018)
The American Astronaut (2001)
American Beauty (1927 film)
American Beauty (1999 film)
The American Beauty (1916)
American Boy: A Profile of Steven Prince (1978)
American Buffalo (1996)
An American Carol (2008)
American Casino (2009)
An American Christmas Carol (1979) (TV)
An American Crime (2007)
American Desi (2001)
American Dharma (2018)
American Dog (2008)
American Dream (1990)
An American Dream (1966)
The American Dreamer (1971)
American Dreamz (2006)
The American Epic Sessions (2017)
American Fable (2016)
American Flyers (1985)
The American Friend (1977)
American Friends (1991)
American Gangster (2007)
The American Gangster (1992)
American Gigolo (1980)
American Girl (2002)
American Girl (2021)
American Girls (1918)
American Gothic (1988 film)
American Gothic (2007 film)
American Graffiti (1973)
American Guerrilla in the Philippines (1950)
American Gun (2002 film)
American Gun (2005 film)
American Hardcore (

Animal (2005 film)
Animal (2014 film)
The Animal (2001)
Animal Crackers (1930 film)
Animal Crackers (2017 film)
Animal Factory (2000)
Animal Farm (1954 film)
Animal Farm (1999 film)
Animal House (1978)
Animal Kingdom (2010)
The Animal Kingdom (1932)
Animals Are Beautiful People (1974)
Animalympics (1980)
The Animatrix series:\nBeyond (2003)\nA Detective Story (2003)\nFinal Flight of the Osiris (2003)\nKid\'s Story (2003)\nMatriculated (2003)\nProgram (2003)\nThe Second Renaissance (2003)\nWorld Record (2003)
Beyond (2003)
A Detective Story (2003)
Final Flight of the Osiris (2003)
Kid\'s Story (2003)
Matriculated (2003)
Program (2003)
The Second Renaissance (2003)
World Record (2003)
Anjaam (1994)
Anjaane (2000 film)
Anjaane (2005 film)
Anjali (1977 film)
Anjali (1990 film)
Anjaneya (2003)
Ankahee (1985 film)
Ankahee (2006 film)
Ankuram (1993)
Anmol (1993)
Ankoku Joshi (2017)
Anna (1951 film)
Anna (1964 film)
Anna (1967 film)
Anna (1987 film)
Anna and the Apocalypse (2018)
Anna Christie

Around the World in 80 Days (1988 film)
Around the World in 80 Days (2004 film)
Around the World in 80 Minutes with Douglas Fairbanks (1931)
Around the World Under the Sea (1966)
Arranged (2007)
The Arrangement (1969)
Arrebato (1980)
Arresting Gena (1997)
Arrietty (2010)
Arrival (2016)
The Arrival (1991 film)
The Arrival (1996 film)
The Arrival of Averill (1992)
The Arrival from the Darkness (1921)
Arriv\xc3\xa9e d\'un train en gare de la Ciotat (1895)
Arriv\xc3\xa9e d\'un train gare de Vincennes (1896)
L\'Arroseur (1896)
L\'Arroseur Arros\xc3\xa9 (1895)
Arrowsmith (1931)
Arsenal (1929)
Arsenal (2017)
The Arsenal Stadium Mystery (1940)
Arsenic and Old Lace (1944)
The Arsonist (1995)
Ars\xc3\xa8ne Lupin (2004)
Art & Copy (2009)
Art Deco Detective (1994)
Art of the Devil
Art of the Devil 2
Art of the Devil 3
Art of the Devil (2004)
Art of the Devil 2 (2005)
Art of the Devil 3 (2008)
Art of Fighting (2006)
The Art of Getting By (2011)
Art Museum by the Zoo (1998)
The Art of Racing in the 

Attica (1980) (TV)
Attilas \'74 (1975)
Attraction (2017)
Au Hasard Balthazar (1966)
Au Pair (film)
Au Pair II
Au Pair 3: Adventure in Paradise
Au Pair (1999)
Au Pair II (2001)
Au Pair 3: Adventure in Paradise (2009)
Au Revoir, Les Enfants (1987)
Au Revoir, UFO (2004)
L\'Auberge Espagnole (2002)
Audace colpo dei soliti ignoti (1959)
Audition (1999 film)
Audition (2005 film)
Audition (2007 film)
The Audition (2000 film)
The Audition (2015 film)
Auditions (1978)
Audrey Rose (1977)
Auggie Rose (2000)
August (1996 film)
August (2008 film)
August (2011 film)
August 32nd on Earth (1998)
August Rush (2006)
August Underground
August Underground's Mordum
August Underground's Penance
August Underground (2001)
August Underground\'s Mordum (2003)
August Underground\'s Penance (2007)
August: Osage County (2013)
Augustin (1995)
Auntie Mame (1958)
El Aura (2005)
Aurangzeb (2013)
Aurora: Operation Intercept (1995)
Aurore (2005)
Auschwitz (2011)
Austin Powers series:\nAustin Powers in Goldmember (2002)\

In [265]:
# movie_data

In [283]:
df = pd.DataFrame(movie_data)

In [290]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2364 entries, 0 to 2363
Columns: 127 entries, AlphaURL to Theme park attraction(s)
dtypes: object(127)
memory usage: 11.2 MB


In [292]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2364 entries, 0 to 2363
Columns: 127 entries, AlphaURL to Theme park attraction(s)
dtypes: object(127)
memory usage: 2.3+ MB


In [293]:
df.columns

Index(['AlphaURL', 'Title', 'MovieURL', 'Image', 'Directed by', 'Written by',
       'Produced by', 'Starring', 'Cinematography', 'Edited by',
       ...
       'Voices of', 'Original title', 'Screen story by', 'Marathi',
       'Inuktitut', 'Korean', 'Backgrounds by', 'Book(s)', 'Original music',
       'Theme park attraction(s)'],
      dtype='object', length=127)